In [ ]:
%run model.ipynb
%run dataloader.ipynb
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import time
import matplotlib.pyplot as plt

In [ ]:
# Entrenamiento
def train(model, epochs,data_loader,criterion,optimizer,cuda=False):
        x = np.arange(1, epochs + 1)
        y = np.empty(epochs)
        start = time.time()
        
        for epoch in range(epochs):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                if (cuda == True):
                    inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with autocast():
                    outputs = model.forward(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            y[epoch] = running_loss
            
        end = time.time()
        print("El entrenamiento tomó " + str(end - start) + " segundos.")
        # Grafica loss
        plt.plot(x, y)
        plt.xlabel('Número de epochs')
        plt.ylabel('Error')
        plt.show

In [ ]:
# Cargamos datos de entrenamiento
labels = 'class_labels_indices.csv'
audio_conf = {'num_mel_bins': 128, 'target_length': 1024, 'freqm': 24, 'timem': 192, 'mixup': 0.5}
data = AudiosetDataset('test_data.json', audio_conf, label_csv = labels)
train_loader = torch.utils.data.DataLoader(data,batch_size=8,
                                          shuffle=True, num_workers=2)

In [ ]:
# Creamos y entrenamos modelo
modelo_audio = ASTModel(class_n = 201)
modelo_audio = modelo_audio.to(device)
entrenables = [p for p in modelo_audio.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(entrenables, 0.001, weight_decay=5e-7, betas=(0.95, 0.999))
criterio = nn.CrossEntropyLoss()
#Entrenamos la red durante 50 pasos, con entropia cruzada y el optimizador ADAM
train(modelo_audio, 2,train_loader,criterio,optimizer,cuda=True)
# Guardar modelo
PATH = './modelo_audio.pth'
torch.save(modelo_audio.state_dict(), PATH)

In [ ]:
# Recomiendo ejecutar esto en lugar del entrenamiento
PATH = './modelo_audio.pth'
modelo_audio = ASTModel(class_n = 201)
modelo_audio.load_state_dict(torch.load(PATH))
modelo_audio = modelo_audio.to(device)

In [ ]:
# Funciones para evaluación
def contarCorrectas(net,batch,labels,func=None):
    '''Dado un batch y sus etiquetas, cuenta el numero de respuestas
    correctas de una red, el parametro func aplica una modificacion al 
    tensor que contiene los datos'''
    salidas=net(batch)
    cantidadCorrectas = 0
    for (output, label) in zip(salidas, labels):
        print(output)
        print(label)
        if torch.argmax(output) == torch.argmax(label):
            cantidadCorrectas = cantidadCorrectas + 1
    return cantidadCorrectas
    
def calcularPrecisionGlobal(net,data_loader,batch_size,cuda=False):
    '''Calcula la precision de una red dado un data_loader,
    recive una funcion que transforma los datos en caso de ser necesario'''
    correctas=0
    for (images,labels) in data_loader:
        if(cuda and torch.cuda.is_available()):
            images=images.cuda()
            labels=labels.cuda()
        correctas+=contarCorrectas(net,images,labels)
    return (100*correctas)/(len(data_loader)*batch_size) 

In [ ]:
precision = calcularPrecisionGlobal(modelo_audio,train_loader,8, cuda = True)
print("Precision del modelo: %.4f%%"%(precision))